# Part 3 - Data Prep

https://www.udemy.com/course/feature-engineering-for-machine-learning

* Types and characteristics of data
* Missing data imputation
* Categorical encoding
* Variable transformation
* Discretization
* Outliers
* Datetime
* Scaling
* Feature creation

## Load Data

In [ ]:
import pandas as pd

df = pd.read_csv('created_raw_data.csv')
print(df.shape)
print(df.info())
df.head()

(1010, 41)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   binary                 1010 non-null   object 
 1   given_name             810 non-null    object 
 2   surname                907 non-null    object 
 3   date_of_birth          908 non-null    object 
 4   phone_number           960 non-null    object 
 5   email                  909 non-null    object 
 6   address                621 non-null    object 
 7   city                   445 non-null    object 
 8   state                  446 non-null    object 
 9   zipcode                845 non-null    float64
 10  duplicate_2            1010 non-null   float64
 11  correlated w target 2  707 non-null    float64
 12  duplicate_1            1010 non-null   float64
 13  uniform corr 1         808 non-null    float64
 14  random choice 2        710 non-null    object

,binary,given_name,surname,date_of_birth,phone_number,email,address,city,state,zipcode,...,constant_2,random label num 12,multicollinearity 1,semi_constant_1,pd qcut1,informative_1,random label num 3,corr_feature_class,semi_constant_2,class
0,binary_2,Thomas,NaN,1925-12-26,291.912.1249x52286,scottbond@example.net,58707 James Meadow Apt. 346,Tiffanyside,NaN,51206.0,...,constant_value,label num hi 10,-0.531847,q_const_0,Low,-0.999102,label num lo 2,0.101048,q_const_0,1
1,binary_1,Melissa,NaN,1972-08-16,(513)806-5749x01300,jonathon55@example.org,NaN,NaN,WV,34775.0,...,constant_value,label num hi 5,-0.345348,q_const_0,High,1.246686,label num lo 3,9.055727,q_const_0,1
2,binary_2,Mark,Scott,1933-02-22,(871)397-5872,shafferamanda@example.org,NaN,NaN,NaN,NaN,...,constant_value,label num hi 4,-1.976924,q_const_0,High,0.962777,label num lo 2,6.433520,q_const_0,1
3,binary_2,Christopher,Patterson,2020-05-31,437-681-7960,heather48@example.com,NaN,NaN,NaN,NaN,...,constant_value,label num hi 4,-0.027502,q_const_0,High,-2.957441,label num lo 2,11.233637,q_const_0,1
4,binary_2,Kent,Thomas,NaN,(294)825-9751x12385,moyerchad@example.com,2558 Clinton Square Suite 741,Ashleyberg,IN,61466.0,...,constant_value,label num hi 1,0.042751,q_const_0,High,1.141165,label num lo 1,8.969995,q_const_0,1


In [ ]:
import preppy.utils as utils
from preppy.version import __version__ as _version_


print(_version_)

utils.report.write_report(df, thresh=.5)

PrepPy Version: 0.1.0
REPORT FOR DATA PREP

#################################################
Columns with Constant Values
#################################################
['constant_1', 'constant_2']

#################################################
Columns with Quasi-Constant Values
#################################################
['binary', 'class', 'constant_1', 'constant_2']

#################################################
Duplicate Rows
#################################################
20

#################################################
Duplicate Columns
#################################################
['informative_2', 'informative_1', 'constant_2']

#################################################
Variables with Noticeably Higher Scales
#################################################
Features with Noticeably Higher Scales (Based on Standard Deviation):
zipcode             29130.432162
standard scaling     9758.774338
Name: std, dtype: float64

Features with Noticeabl

In [ ]:
import preppy.utils as preppy

consts = preppy.functions.identify_consts(df)
quasi_consts = preppy.functions.identify_quasi_consts(df)
duplicates = preppy.functions.check_col_duplicates(df)
print(duplicates)
print(consts)
print(quasi_consts)

['informative_2', 'informative_1', 'constant_2']
['constant_1', 'constant_2']
['constant_1', 'constant_2']


In [ ]:
# numeric_df = df.apply(pd.to_numeric, errors='coerce')
all_deletes = list(set(consts + quasi_consts + duplicates))
for col in all_deletes:
  print(col, df[col].dtype)
  if df[col].dtype in ['float64', 'int64']:
    df_numerical.remove(col)
  elif df[col].dtype in ['object']:
    df_object.remove(col)
    df_categorical_features.remove(col)
  else:
    df_discreet.remove(col)


constant_2 object
informative_1 float64
constant_1 object
informative_2 float64


## PrepPy Pipeline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import preppy.utils as preppy

pipe = [
    ('constants', preppy.classes.RemoveConstants()),
    ('quasiconsts', preppy.classes.RemoveQuasiConstants(thresh=0.8)),
    ('duplicates', preppy.classes.DropDuplicates()),
    ('missing', preppy.classes.HandleMissingValues()),
    # ('encoding', HandleCatEncodeing())
]

pipe_model = Pipeline(pipe)
data = pipe_model.fit_transform(df)
cols = [col for col in df.columns if col not in consts + quasi_consts + duplicates]
nu_df = pd.DataFrame(data, columns=cols)
nu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   binary                 1000 non-null   object
 1   given_name             1000 non-null   object
 2   surname                1000 non-null   object
 3   date_of_birth          1000 non-null   object
 4   phone_number           1000 non-null   object
 5   email                  1000 non-null   object
 6   address                1000 non-null   object
 7   city                   1000 non-null   object
 8   state                  1000 non-null   object
 9   zipcode                1000 non-null   object
 10  duplicate_2            1000 non-null   object
 11  correlated w target 2  1000 non-null   object
 12  duplicate_1            1000 non-null   object
 13  uniform corr 1         1000 non-null   object
 14  random choice 2        1000 non-null   object
 15  standard scaling      

/content/preppy/utils/functions.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfx[feat] = df[feat].fillna(df[feat].mode()[0])


In [ ]:
import pickle

# Load the pickled variable from the file
with open('var_types.pkl', 'rb') as f:
    var_types = pickle.load(f)

print(var_types)

{'df_numerical': ['zipcode', 'duplicate_2', 'correlated w target 2', 'duplicate_1', 'uniform corr 1', 'standard scaling', 'correlated w target 1', 'target', 'min max scaling', 'multicollinearity 4', 'informative_2', 'multicollinearity 2', 'outliers 2', 'uniform corr 2', 'multicollinearity 3', 'outliers 1', 'multicollinearity 1', 'informative_1', 'corr_feature_class', 'class'], 'df_object': ['binary', 'given_name', 'surname', 'date_of_birth', 'phone_number', 'email', 'address', 'city', 'state', 'random choice 2', 'constant_1', 'random choice 4', 'pd qcut3', 'pd qcut2', 'random choice 7', 'constant_2', 'random label num 12', 'semi_constant_1', 'pd qcut1', 'random label num 3', 'semi_constant_2'], 'df_discreet': [], 'df_categorical_features': ['binary', 'given_name', 'surname', 'date_of_birth', 'phone_number', 'email', 'address', 'city', 'state', 'random choice 2', 'constant_1', 'random choice 4', 'pd qcut3', 'pd qcut2', 'random choice 7', 'constant_2', 'random label num 12', 'semi_consta

In [ ]:
df_numerical = var_types['df_numerical']
df_object = var_types['df_object']
df_discreet = var_types['df_discreet']
df_categorical_features = var_types['df_categorical_features']

In [ ]:
# code along
df_numerical = [col for col in nu_df.columns if col in var_types['df_numerical']]
nu_df[df_numerical] = nu_df[df_numerical].astype(float)
nu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   binary                 1000 non-null   object 
 1   given_name             1000 non-null   object 
 2   surname                1000 non-null   object 
 3   date_of_birth          1000 non-null   object 
 4   phone_number           1000 non-null   object 
 5   email                  1000 non-null   object 
 6   address                1000 non-null   object 
 7   city                   1000 non-null   object 
 8   state                  1000 non-null   object 
 9   zipcode                1000 non-null   float64
 10  duplicate_2            1000 non-null   float64
 11  correlated w target 2  1000 non-null   float64
 12  duplicate_1            1000 non-null   float64
 13  uniform corr 1         1000 non-null   float64
 14  random choice 2        1000 non-null   object 
 15  stand

## Feature Engineering

### Feature Combination

In [ ]:
# create a new variable by combining two variables
df['scaling_combined'] = df['standard scaling'] + df['min max scaling']
df.drop(['standard scaling', 'min max scaling'], axis=1, inplace=True)

### Categorical Encoding

In [ ]:
# code along
import preppy.utils as utils

# Impute missing values before applying do_OHE
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

df = utils.functions.do_OHE(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   binary                             1010 non-null   int64  
 1   given_name                         1010 non-null   float64
 2   surname                            1010 non-null   float64
 3   date_of_birth                      1010 non-null   float64
 4   phone_number                       1010 non-null   float64
 5   email                              1010 non-null   float64
 6   address                            1010 non-null   float64
 7   city                               1010 non-null   float64
 8   state                              1010 non-null   float64
 9   zipcode                            845 non-null    float64
 10  duplicate_2                        1010 non-null   float64
 11  correlated w target 2              707 non-null    float

In [ ]:
df.to_csv('prepared_data.csv', index=False)